In [1]:
import time
import numpy as np
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from xgboost.sklearn import XGBClassifier

from tianchi.user_location_prediction.pysrc.feature import *
from tianchi.user_location_prediction.pysrc.load import *

/home/zhouzr/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
sample = load_sample()
shop = load_shop()
sample_wifi = load_sample_wifi()

/home/zhouzr/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
def construct_batch(sample_df, sample_wifi_df, shop_df):
    feat_list = []

    weekday_feat = weekday_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(weekday_feat)

    hour_feat = hour_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(hour_feat)

    spaceloc_feat = spaceloc_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(spaceloc_feat)

    spaceloc_dist_feat = spaceloc_dist_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(spaceloc_dist_feat)

    wifi_power_feat = wifi_power_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(wifi_power_feat)

    wifi_powerloc_dist_feat = wifi_powerloc_dist_featext(sample_df, sample_wifi_df, shop_df, wifi_power_feat)
    feat_list.append(wifi_powerloc_dist_feat)

    wifi_flag_feat = wifi_flag_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(wifi_flag_feat)

    wifi_flagloc_dist_feat = wifi_flagloc_dist_featext(sample_df, sample_wifi_df, shop_df, wifi_flag_feat)
    feat_list.append(wifi_flagloc_dist_feat)

    feat = np.concatenate(feat_list, axis=1)
    train_idx = sample_df[sample_df.row_id.isnull()].index.tolist()[-1] + 1

    train_x = feat[:train_idx]
    train_y = sample_df[sample_df.row_id.isnull()]['shop_id'].values
    test_x = feat[train_idx:]
    test_row_id = list(sample_df[-sample_df.row_id.isnull()]['row_id'].values)

    return train_x, train_y, test_x, test_row_id

In [23]:
import time
t = time.time()
mall_list = sample.mall_id.unique()
submission_row_id = []
submission_shop = []
acc_list = []
for mall_i in mall_list:
    model = LogisticRegression()
    
    sample_i = sample[sample.mall_id==mall_i].copy().reset_index(drop=True)
    sample_wifi_i = sample_wifi[sample_wifi.sample_id.isin(sample_i.sample_id.values)].copy().reset_index(drop=True)
    shop_i = shop[shop.mall_id==mall_i].copy().reset_index(drop=True)
    
    train_x, train_y, test_x, test_row_id = construct_batch(sample_i, sample_wifi_i, shop_i)
    model.fit(train_x, train_y)
    
    acc = model.score(train_x, train_y)
    print(acc)
    shop_pre = model.predict(test_x)
    
    submission_row_id.extend(test_row_id)
    submission_shop.extend(shop_pre)
    acc_list.append(acc)
print('cost %.2f min'%((time.time()-t)/60))  

0.960879463216
0.770260439401
0.927787128316
0.919704366075
0.84002999162
0.787987713221
0.858692493947
0.900395286396
0.962273857498
0.728931891482
0.901712822155
0.962219598583
0.804472242662
0.966216698807
0.960797397614
0.841556487155
0.968502451905
0.841202872531
0.947966852958
0.861816175311
0.802233266637
0.908436886854
0.910732617825
0.700689235759
0.966236955187
0.979636237055
0.965564029383
0.934032983508
0.952572420726
0.979309435952
0.932010953049
0.928238527725
0.97557280818
0.924155695478
0.969344141489
0.842630346144
0.866880513232
0.956352636128
0.953641576186
0.961255656109
0.960143841774
0.951461831806
0.914800389484
0.808098045299
0.752278096524
0.963106796117
0.809703766915
0.970686007857
0.978177150193
0.959144542773
0.953746085281
0.928549599658
0.911251015435
0.908277404922
0.977483844911
0.936798803291
0.956712986104
0.979861111111
0.957969579696
0.927162906378
0.755777310924
0.949586556415
0.957881873727
0.960690316395
0.936826568266
0.897448936351
0.9434678981

In [24]:
print(np.array(acc_list).mean())

0.921879607585


In [25]:
import os
path = '/home/zhouzr/data/tianchi/user_location_predict'
os.chdir(path)
submission = pd.DataFrame({'row_id':submission_row_id, 'shop_id':submission_shop}, 
                          columns=['row_id','shop_id'])
submission['row_id'] = submission['row_id'].astype('int').astype('str')
submission.to_csv('./submission/result_2.csv', index=None, sep=',')

In [26]:
print(np.array(acc_list).min())
print(np.array(acc_list).max())

0.700689235759
0.984816196058


In [27]:
submission.shape

(483931, 2)